# 価格は正しいです

今日、商品の価格を推定するためのより複雑なソリューションを構築しています。

1。Day2.0ノートブック：400,000のトレーニングデータを使用してRAGデータベースを作成する
2。Day2.1ノートブック：2dで視覚化します
3。2.2日目ノートブック：3Dで視覚化します
4。Day2.3ノートブック：GPT-4O-MINIでRAGパイプラインを構築してテストする
5。Day2.4ノートブック：（a）ランダムフォレストプライザー（b）すべての価格からの寄付を可能にするアンサンブルプライザーを作成する

うーん！それは一日で乗り越えるのにたくさんのことです！

＃＃ ご注意ください：

私たちはすでに、独自のファインチューニングされたLLMを使用して非常に強力な製品推定器を持っています。ほとんどの人はそれに非常に満足するでしょう！これらの追加の手順を追加する主な理由は、RAGとエージェントワークフローで専門知識を深めることです。

##ランダムフォレストとアンサンブルで仕上げます

In [ ]:
# 輸入

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from items import Item
from testing import Tester
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import joblib


In [ ]:
# 定数

QUESTION = "How much does this cost to the nearest dollar?\n\n"
DB = "products_vectorstore"

In [ ]:
# 環境

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# Huggingfaceにログインします

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
# 顔を抱き締めるためにログインした後の別の輸入品 - ありがとう、Trung N.！

from items import Item

In [ ]:
# テストピクルスファイルにロード：

with open('test.pkl', 'rb') as file:
    test = pickle.load(file)

In [ ]:
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')

In [ ]:
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
prices = [metadata['price'] for metadata in result['metadatas']]

# ランダムフォレスト

これで、ランダムな森林モデルを訓練します。

6週目としたこととの違いを見つけることができますか？ 6週目には、Word2Vecモデルを使用してベクターを形成しました。今回は、Sentencetransformerモデルから、すでにChromaにあるベクトルを使用します。

In [ ]:
# この次の行は私のM1 Macで1時間かかります！

rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(vectors, prices)

In [ ]:
# モデルをファイルに保存します

joblib.dump(rf_model, 'random_forest_model.pkl')

In [ ]:
# もう一度ロードします

rf_model = joblib.load('random_forest_model.pkl')

In [ ]:
from agents.specialist_agent import SpecialistAgent
from agents.frontier_agent import FrontierAgent
from agents.random_forest_agent import RandomForestAgent

In [ ]:
specialist = SpecialistAgent()
frontier = FrontierAgent(collection)
random_forest = RandomForestAgent()

In [ ]:
def description(item):
    return item.prompt.split("to the nearest dollar?\n\n")[1].split("\n\nPrice is $")[0]

In [ ]:
def rf(item):
    return random_forest.price(description(item))

In [ ]:
Tester.test(rf, test)

In [ ]:
product = "Quadcast HyperX condenser mic for high quality audio for podcasting"

In [ ]:
print(specialist.price(product))
print(frontier.price(product))
print(random_forest.price(product))

In [ ]:
specialists = []
frontiers = []
random_forests = []
prices = []
for item in tqdm(test[1000:1250]):
    text = description(item)
    specialists.append(specialist.price(text))
    frontiers.append(frontier.price(text))
    random_forests.append(random_forest.price(text))
    prices.append(item.price)

In [ ]:
mins = [min(s,f,r) for s,f,r in zip(specialists, frontiers, random_forests)]
maxes = [max(s,f,r) for s,f,r in zip(specialists, frontiers, random_forests)]

X = pd.DataFrame({
    'Specialist': specialists,
    'Frontier': frontiers,
    'RandomForest': random_forests,
    'Min': mins,
    'Max': maxes,
})

# yをシリーズに変換します
y = pd.Series(prices)

In [ ]:
# 線形回帰を訓練します
np.random.seed(42)

lr = LinearRegression()
lr.fit(X, y)

feature_columns = X.columns.tolist()

for feature, coef in zip(feature_columns, lr.coef_):
    print(f"{feature}: {coef:.2f}")
print(f"Intercept={lr.intercept_:.2f}")

In [ ]:
joblib.dump(lr, 'ensemble_model.pkl')

In [ ]:
from agents.ensemble_agent import EnsembleAgent
ensemble = EnsembleAgent(collection)

In [ ]:
ensemble.price(product)

In [ ]:
def ensemble_pricer(item):
    return max(0,ensemble.price(description(item)))

In [ ]:
Tester.test(ensemble_pricer, test)

# なんて一日！

フランターのぼろきれパイプライン、トランスベースのエンコーディングを使用したランダムフォレストモデル、アンサンブルモデルなど、多くのことができました。

確かにあなたはもっとうまくやることができます！

これをファインチューニングし、コンポーネントをアンサンブルに追加して、パフォーマンスを打ち負かしてみてください。